# Video Game Sales

### The global video game industry is a billion-dollar business and has been for many years. In 2022, the revenue from the worldwide gaming market was estimated at almost 347 billion U.S. dollars, with the mobile gaming market generating an estimated 248 billion U.S. dollars of the total. (https://www.statista.com/topics/868/video-games/)   

#### By analyzing variables connected to previously released games, there might be a possibility to train a machine learning model to help predict future sales, and also what kind of games would generate the most money.

In [17]:
import pandas as pd

In [5]:
# https://www.kaggle.com/datasets/rush4ratio/video-game-sales-with-ratings/data
video_games_sales=pd.read_csv("Video_Games_Sales_as_at_22_Dec_2016.csv")

# EDA

In [14]:
# Global_Sales will be the predictor variable in this case.
# The other sales columns might be too similar/connected to Global_Sales and create overfitting? 
video_games_sales.head()

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN


### Explaining the column names

From the data info:

Fields include:
   
    Name - The games name    
    Platform - Platform of the games release (i.e. PC,PS4, etc.)    
    Year - Year of the game's release
    Genre - Genre of the game
    Publisher - Publisher of the game
    NA_Sales - Sales in North America (in millions)
    EU_Sales - Sales in Europe (in millions)
    JP_Sales - Sales in Japan (in millions)
    Other_Sales - Sales in the rest of the world (in millions)
    Global_Sales - Total worldwide sales.
    Critic_score - Aggregate score compiled by Metacritic staff
    Critic_count - The number of critics used in coming up with the Critic_score
    User_score - Score by Metacritic's subscribers
    User_count - Number of users who gave the user_score
    Developer - Party responsible for creating the game
    Rating - The ESRB ratings


In [16]:
# There seems to be missing values, especially at the score columns. 
# Column 13, User_Count, is missing more than 50% of the total values.
# Strings are in object type, which might be ok?
# User_Score are of type object and not float like Critics_Score.
video_games_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16719 entries, 0 to 16718
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16717 non-null  object 
 1   Platform         16719 non-null  object 
 2   Year_of_Release  16450 non-null  float64
 3   Genre            16717 non-null  object 
 4   Publisher        16665 non-null  object 
 5   NA_Sales         16719 non-null  float64
 6   EU_Sales         16719 non-null  float64
 7   JP_Sales         16719 non-null  float64
 8   Other_Sales      16719 non-null  float64
 9   Global_Sales     16719 non-null  float64
 10  Critic_Score     8137 non-null   float64
 11  Critic_Count     8137 non-null   float64
 12  User_Score       10015 non-null  object 
 13  User_Count       7590 non-null   float64
 14  Developer        10096 non-null  object 
 15  Rating           9950 non-null   object 
dtypes: float64(9), object(7)
memory usage: 2.0+ MB


In [22]:
# Missing values:
# Name, Year_of_Release and other columns with smaller count might have their rows dropped.
# The score columns are not easily dropped without risking an underfitted model.
video_games_sales.isna().sum()

Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Other_Sales           0
Global_Sales          0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Developer          6623
Rating             6769
dtype: int64

In [24]:
# What score values are missing, and why?
# (Något sätt att kolla upp värdena, barchart eller ngn annan sammanfattning. Varför saknas hälften, och varför just de?)